# Data preprocessing and pipelines


In [1]:
# General imports
%matplotlib inline
import pandas as pd
import openml as oml
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,RandomForestClassifier,GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR,SVC
from tqdm.notebook import tqdm as tqdm

In [2]:
data = pd.read_csv("carprice.csv")
data.drop(columns=["car_ID", "symboling"], inplace=True)
X = data.drop(columns=["price"])
y = data["price"]

In [3]:
X.head()

,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg
0,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27
1,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27
2,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26
3,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30
4,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22


In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CarName           205 non-null    object 
 1   fueltype          205 non-null    object 
 2   aspiration        205 non-null    object 
 3   doornumber        205 non-null    object 
 4   carbody           205 non-null    object 
 5   drivewheel        205 non-null    object 
 6   enginelocation    205 non-null    object 
 7   wheelbase         205 non-null    float64
 8   carlength         205 non-null    float64
 9   carwidth          205 non-null    float64
 10  carheight         205 non-null    float64
 11  curbweight        205 non-null    int64  
 12  enginetype        205 non-null    object 
 13  cylindernumber    205 non-null    object 
 14  enginesize        205 non-null    int64  
 15  fuelsystem        205 non-null    object 
 16  boreratio         205 non-null    float64
 1

In [5]:
y.head()

0    13495.0
1    16500.0
2    16500.0
3    13950.0
4    17450.0
Name: price, dtype: float64

## Exercise 1: Build a pipeline

Implement a function `build_pipeline` that does the following:
- Impute missing values by replacing NaN's with the feature median for numerical features.
- Encode the categorical features using OneHotEncoding.
- Scale the data using min-max scaling.
- Attach the given regression model to the end of the pipeline

In [5]:
def build_pipeline(regressor, categorical):
    cat_pipe = make_pipeline(OneHotEncoder(handle_unknown='ignore'))
    num_pipe = make_pipeline(SimpleImputer(strategy='mean'))
    num_pipe.steps.insert(1,["scaler", MinMaxScaler()]) 
    transform = make_column_transformer((cat_pipe, categorical), remainder=num_pipe)
    return Pipeline(steps=[('preprocess', transform), ('reg', regressor)])

## Exercise 2: Test the pipeline
Test the pipeline by evaluating linear regression (without scaling) on the dataset, using 5-fold cross-validation and $R^2$. 

In [8]:
categorical = X.select_dtypes(include=["object"]).columns
regressor = LinearRegression()
pipe = build_pipeline(regressor,categorical)
scores = cross_val_score(pipe, X, y)
print(scores)

[ 0.30887817  0.86510307 -1.24922953  0.207156   -1.47980303]


In [9]:
print(scores.mean())

-0.26957906160261846


## Exercise 3: A first benchmark
Evaluate the following algorithms in their default settings, both with and without scaling, and interpret the results:  
- Linear regression
- Ridge
- Lasso
- SVM (RBF)
- RandomForests
- GradientBoosting

In [10]:
models = [LinearRegression(), Ridge(), Lasso(), RandomForestRegressor(), GradientBoostingRegressor(), SVR()]
for m in tqdm(models): # nstantly make your loops show a smart progress meter
    pipe = build_pipeline(m,categorical)
    scores = cross_val_score(pipe, X, y)
    print("R^2 score for {}: {:.2f}".format(m.__class__.__name__, scores.mean()))

  0%|          | 0/6 [00:00<?, ?it/s]

R^2 score for LinearRegression: -0.27
R^2 score for Ridge: 0.40


/Users/orperets/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4016858.5758827627, tolerance: 1207668.5476894265
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/orperets/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3431548.189468965, tolerance: 1033374.6192783266
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/orperets/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1384109.366284132, tolerance: 1227640.9712946396
  model = cd_fast.sparse_enet_coordinate_descent(


R^2 score for Lasso: 0.36
R^2 score for RandomForestRegressor: 0.40
R^2 score for GradientBoostingRegressor: 0.55
R^2 score for SVR: -0.20


## Exercise 4: Classification

1. Use the original dataset and convert the target columns to 0 (cheap) / 1 (expensive)
2. Re-evaluate the pipeline. What are the conclusions?

In [14]:
mean = data["price"].mean()
data.loc[data["price"] < mean, "price"] = 0
data.loc[data["price"] >= mean, "price"] = 1


,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,1.0
1,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,1.0
2,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,1.0
3,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,1.0
4,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,1.0
201,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,1.0
202,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,1.0
203,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,...,145,idi,3.01,3.40,23.0,106,4800,26,27,1.0


In [15]:
X = data.drop(columns=["price"])
y = data["price"]

In [17]:
models = [RandomForestClassifier(),GradientBoostingClassifier(), SVC()]
for m in tqdm(models):
    pipe = build_pipeline(m,categorical)
    scores = cross_val_score(pipe, X, y)
    print("accuracy for {}: {:.2f}".format(m.__class__.__name__, scores.mean()))

  0%|          | 0/3 [00:00<?, ?it/s]

accuracy for RandomForestClassifier: 0.88
accuracy for GradientBoostingClassifier: 0.85
accuracy for SVC: 0.86


# Exploring the GridSearchCV Class

In [ ]:
# Exploring the GridSearchCV Class
GridSearchCV(
    estimator=,     # A sklearn model
    param_grid=,    # A dictionary of parameter names and values
    cv=,            # An integer that represents the number of k-folds
    scoring=,       # The performance measure (such as r2, precision)
    n_jobs=,        # The number of jobs to run in parallel
)

In [18]:
knn = KNeighborsRegressor()
X = pd.get_dummies(X)

params = {
    'n_neighbors': [3,5,7,9,11,13],
    'weights': ['uniform', 'distance'],
    'p': [1,2] # the power of Minikowski
}

clf = GridSearchCV(
    estimator=knn,
    param_grid=params,
    cv=5,
    n_jobs=5,
)

clf.fit(X, y)
print(clf.best_params_)

{'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
